## Stable Diffusion

In [1]:
! python -m pip install --upgrade pip
! pip install botocore --upgrade --quiet

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.5 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.71 requires botocore==1.29.71, but you have botocore 1.29.94 which is incompatible.


In [2]:
!pip install "sagemaker==2.116.0" "huggingface_hub==0.13.1" --upgrade --quiet

In [5]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123
sagemaker bucket: sagemaker-us-west-2-687912291502
sagemaker session region: us-west-2


In [8]:
%%writefile code/inference.py
import base64
import torch
from io import BytesIO
from diffusers import StableDiffusionPipeline,DiffusionPipeline
import deepspeed


def model_fn(model_dir):

    # Load stable diffusion and move it to the GPU
    pipe = StableDiffusionPipeline.from_pretrained(model_dir, torch_dtype=torch.float16, revision="fp16")
    pipe=deepspeed.init_inference(
        model=getattr(pipe,"model", pipe),      # Transformers models
        mp_size=1,        # Number of GPU
        dtype=torch.float16, # dtype of the weights (fp16)
        replace_method="auto", # Lets DS autmatically identify the layer to replace
        replace_with_kernel_inject=False, # replace the model with the kernel injector
    )

    print("!!!!DeepSpeed Inference Engine initialized!!!!!!!!")
    pipe = pipe.to("cuda")
    torch.cuda.synchronize("cuda")
    return pipe


def predict_fn(data, pipe):

    # get prompt & parameters
    prompt = data.pop("inputs", data)
    print(prompt)
    # set valid HP for stable diffusion
    num_inference_steps = data.pop("num_inference_steps", 50)
    guidance_scale = data.pop("guidance_scale", 7.5)
    num_images_per_prompt = data.pop("num_images_per_prompt", 4)
    width = data.pop("width", 512)
    height = data.pop("height", 512)

    # run generation with parameters
    generated_images = pipe(
        prompt,
        #num_inference_steps=num_inference_steps,
        #guidance_scale=guidance_scale,
        height=height,
        width=width,
        num_images_per_prompt=num_images_per_prompt
    )["images"]

    # create response
    encoded_images = []
    for image in generated_images:
        buffered = BytesIO()
        image.save(buffered, format="JPEG")
        encoded_images.append(base64.b64encode(buffered.getvalue()).decode())

    # create response
    return {"generated_images": encoded_images}


Writing code/inference.py


In [2]:
!pip install -r ./inference/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 11.3 MB/s eta 0:00:0000:01
  Using cached transformers-4.26.1-py3-none-any.whl (6.3 MB)
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Using cached spacy-3.5.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.5 MB)
  Using cached nvgpu-0.9.0-py2.py3-none-any.whl (9.4 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
  Using cached sentencepiece-0.1.97-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.9/709.9 kB 9.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.11.1-py2.py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (145 kB)
  Using cached triton-2.0.0.dev20221120-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.7 MB)
  Using cached deepspeed_mii-0.0.3-py3-none-any.whl (44 kB)
  Using cached clip-0.2.0.tar.gz (5.5 kB)
  Preparing metadata (setup.py) ... done
  Using cached hj

In [17]:
import os
import json
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionImg2ImgPipeline
import boto3
import sagemaker
import uuid
import torch
from torch import autocast
from PIL import Image
import io
import requests
import traceback
import os
import json
import torch
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionImg2ImgPipeline
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import deepspeed



#model_dir='/root/dreambooth/models_diffuser/'
model_dir='runwayml/stable-diffusion-v1-5'
model = StableDiffusionPipeline.from_pretrained(model_dir, torch_dtype=torch.float16, revision="fp16")
print("model loaded:",model)
 
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True


try:
    print("begin load deepspeed....")    
    model=deepspeed.init_inference(
        model=getattr(model,"model", model),      # Transformers models
        mp_size=1,        # Number of GPU
        dtype=torch.float16, # dtype of the weights (fp16)
        replace_method="auto", # Lets DS autmatically identify the layer to replace
        replace_with_kernel_inject=False, # replace the model with the kernel injector
    )
    print('model accelarate with deepspeed!')
except Exception as e:
    print("deepspeed accelarate excpetion!")
    print(e)
    
model = model.to("cuda")
model.enable_attention_slicing()    

model loaded: StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.14.0",
  "feature_extractor": [
    null,
    null
  ],
  "requires_safety_checker": false,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

begin load deepspeed....
[2023-03-08 02:00:33,784] [INFO] [logging.py:68:log_dist] [Rank -1] DeepSpeed info: version=0.7.6, git-hash=unknown, git-branch=unknown
[2023-03-08 02:00:33,785] [WARNING] [config_utils.py:68:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2023-03-08 02:00:33,786] [INFO] [logging.py:68:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups 

### Inference

In [ ]:
from PIL import Image
from io import BytesIO
from IPython.display import display
import base64
import matplotlib.pyplot as plt

# helper decoder
def decode_base64_image(image_string):
  base64_image = base64.b64decode(image_string)
  buffer = BytesIO(base64_image)
  return Image.open(buffer)

# display PIL images as grid
def display_images(images=None,columns=3, width=100, height=100):
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):
        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        plt.axis('off')
        plt.imshow(image)

In [ ]:
%%time
prompt = "A dog trying catch a flying pizza art drawn by disney concept artists, golden colour, high quality, highly detailed, elegant, sharp focus"
#prompt = "portrait photo headshot by mucha, sharp focus, elegant, render, octane, detailed, award winning photography, masterpiece, rim lit"
#prompt = "priest, blue robes, 68 year old nun, national geographic, portrait, photo, photography"
#prompt = "hotel room with a swimming pool outside of the window, TV on the table, moon in the sky"


images = model(prompt, num_images_per_prompt=5, num_inference_steps=25, guidance_scale=9).images
             for image in images:
                bucket, key = get_bucket_and_key(output_s3uri)
                key = '{0}{1}.jpg'.format(key, uuid.uuid4())
                buf = io.BytesIO()
                image.save(buf, format='JPEG')
                s3_client.put_object(
                    Body = buf.getvalue(), 
                    Bucket = bucket, 
                    Key = key, 
                    ContentType = 'image/jpeg'
                )
                print('image: ', 's3://{0}/{1}'.format(bucket, key))

# decode images
decoded_images = [decode_base64_image(image) for image in response["generated_images"]]

# visualize generation
display_images(decoded_images)